In [1]:
from astropy.io import ascii
from pathlib import Path
import matplotlib.pyplot as plt
from cxotime import CxoTime
import numpy as np
from scipy.integrate import cumulative_trapezoid as cumtrapz
import json
from storms import SolarWind

In [2]:
fns = list(Path(".").glob("*.json"))

In [3]:
plt.rc("font", size=18)
plt.rc("axes", linewidth=2)
plt.rcParams['xtick.major.width'] = '2'
plt.rcParams['xtick.minor.width'] = '2'
plt.rcParams['xtick.major.size'] = '6'
plt.rcParams['xtick.minor.size'] = '3'
plt.rcParams['ytick.major.width'] = '2'
plt.rcParams['ytick.minor.width'] = '2'
plt.rcParams['ytick.major.size'] = '6'
plt.rcParams['ytick.minor.size'] = '3'


In [4]:
#fig, ax = plt.subplots(figsize=(10,10))
tts = []
dds = []
tss = []
tms = []
tds = []
triggers = []
for fn in fns:
    info = json.load(open(fn, "r"))
    if info["shutdown"] != "YES":
        continue
    print(fn)
    te = CxoTime(info["oormpen_time"])
    ts = CxoTime(info["shutdown_time"])
    td = CxoTime(info["oormpds_time"])
    browse = "2023" in str(fn)
    sw = SolarWind(te, td, no_hrc=True, browse=browse)
    max_p3 = np.nanmax(sw["p3"])
    idx_max = np.nanargmax(sw["p3"])
    tm = sw.times.secs[idx_max]
    dd = sw["p3"]
    nidxs = ~np.isnan(dd)
    tts.append(sw.times.secs[nidxs])
    dds.append(dd[nidxs])
    tms.append(tm)
    tds.append(td.secs)
    tss.append(ts.secs)
    triggers.append(info["trigger"])
#for tt, dd in zip(tts, dds):
#    ax.plot(tt, dd)
#ax.set_yscale("log")
#ax.set_xlim(-50.0, 500.0)

2021_301.json
2023_113.json
2017_250.json
2022_087.json
2015_173.json
2023_058.json
2023_074.json
2017_254.json


In [22]:
ttypes = ["acis", "hrc", "manual"]
styles = ["-", "--", ":"]

In [23]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20,10))
for i, tt in enumerate(tts):
    dd = dds[i]
    td = tds[i]
    ts = tss[i]
    tm = tms[i]
    trigger = triggers[i]
    ls = styles[ttypes.index(trigger)]
    sidxs = tt >= ts
    ax1.plot((tt-ts)[sidxs]/3600.0, cumtrapz(dd[sidxs], (tt-ts)[sidxs], initial=0.0), 
             lw=2, color=f"C{i}", ls=ls)
    if ts-tm > 0.0:
        midxs = tt >= tm
        ax2.plot((tt-tm)[midxs]/3600.0, cumtrapz(dd[midxs], (tt-tm)[midxs], initial=0.0), 
                lw=2, color=f"C{i}", ls=ls)
        ax2.axvline((ts-tm)/3600.0, ls="--", lw=2, color=f"C{i}")
ax1.plot([0], [0], color="k", label="ACIS", ls="-")
ax1.plot([0], [0], color="k", label="HRC", ls="--")
ax1.plot([0], [0], color="k", label="Manual")
for ax in [ax1, ax2]:
    ax.set_xlabel("Time (hours)")
    ax.set_yscale("log")
ax1.legend()

In [24]:
fig.savefig("flu.png")